## Import libraries

In [1]:
import numpy as np   # import numpy
import pandas as pd  # import pandas
import os
import gc   # for gabage collection
import seaborn as sns  # data visualization lib
import matplotlib.pyplot as plt
import glob 
from scipy import stats
%matplotlib inline

<hr  style= "
  display: block; 
  margin-top: 0.5em; 
  margin-bottom: 0.5em;
  margin-left: auto;
  margin-right: auto;
  border-style: inset;
  border-width: 15px "/>

## Create a few algos for viz so that they can be called anytime


In [3]:
# some customized function for plotting data
def plot_corr(df,cols):
    fig = plt.figure(figsize=(10,8))
    sns.heatmap(df[cols].corr(),cmap='RdBu_r', annot=True, center=0.0)
    plt.show()
    
def plot_count(df,col,fsize,rotation=None,fillna=True):
    fig = plt.figure(figsize=fsize)
    if fillna:
        value_count = df[col].fillna('unknown').value_counts()
    sns.barplot(value_count.index,value_count.values)
    plt.xticks(rotation=rotation)
    plt.title('value counts for {}'.format(col))
    plt.show()

<hr  style= "
  display: block; 
  margin-top: 0.5em; 
  margin-bottom: 0.5em;
  margin-left: auto;
  margin-right: auto;
  border-style: inset;
  border-width: 15px "/>

## Load All Data

In [2]:
def load_data(path): 
    return glob.glob(path+"*.csv")   

In [3]:
df_files = load_data('/Users/mega/concordia_courses/1260_ML/home_credit_risk/data/')
df_files.sort()
df_files

['/Users/mega/concordia_courses/1260_ML/home_credit_risk/data/POS_CASH_balance.csv',
 '/Users/mega/concordia_courses/1260_ML/home_credit_risk/data/application_train.csv',
 '/Users/mega/concordia_courses/1260_ML/home_credit_risk/data/bureau.csv',
 '/Users/mega/concordia_courses/1260_ML/home_credit_risk/data/bureau_balance.csv',
 '/Users/mega/concordia_courses/1260_ML/home_credit_risk/data/credit_card_balance.csv',
 '/Users/mega/concordia_courses/1260_ML/home_credit_risk/data/installments_payments.csv',
 '/Users/mega/concordia_courses/1260_ML/home_credit_risk/data/previous_application.csv']

In [4]:
df_pos_cash = pd.read_csv(df_files[0])
df_app_train = pd.read_csv(df_files[1])
df_burr =pd.read_csv(df_files[2])
df_burr_bal = pd.read_csv(df_files[3])
df_cc_bal = pd.read_csv(df_files[4])
df_inst_pay =  pd.read_csv(df_files[5])
df_prev_app =pd.read_csv(df_files[6])
all_dfs = [df_pos_cash, df_app_train, df_burr, df_burr_bal, df_cc_bal, df_inst_pay, df_prev_app]

<hr  style= "
  display: block; 
  margin-top: 0.5em; 
  margin-bottom: 0.5em;
  margin-left: auto;
  margin-right: auto;
  border-style: inset;
  border-width: 15px "/>

### STATS

#### Data types

In [5]:
for df in all_dfs:
    for col in df:
        print(f'{col}: {df[col].dtype}')

SK_ID_PREV: int64
SK_ID_CURR: int64
MONTHS_BALANCE: int64
CNT_INSTALMENT: float64
CNT_INSTALMENT_FUTURE: float64
NAME_CONTRACT_STATUS: object
SK_DPD: int64
SK_DPD_DEF: int64
SK_ID_CURR: int64
TARGET: int64
NAME_CONTRACT_TYPE: object
CODE_GENDER: object
FLAG_OWN_CAR: object
FLAG_OWN_REALTY: object
CNT_CHILDREN: int64
AMT_INCOME_TOTAL: float64
AMT_CREDIT: float64
AMT_ANNUITY: float64
AMT_GOODS_PRICE: float64
NAME_TYPE_SUITE: object
NAME_INCOME_TYPE: object
NAME_EDUCATION_TYPE: object
NAME_FAMILY_STATUS: object
NAME_HOUSING_TYPE: object
REGION_POPULATION_RELATIVE: float64
DAYS_BIRTH: int64
DAYS_EMPLOYED: int64
DAYS_REGISTRATION: float64
DAYS_ID_PUBLISH: int64
OWN_CAR_AGE: float64
FLAG_MOBIL: int64
FLAG_EMP_PHONE: int64
FLAG_WORK_PHONE: int64
FLAG_CONT_MOBILE: int64
FLAG_PHONE: int64
FLAG_EMAIL: int64
OCCUPATION_TYPE: object
CNT_FAM_MEMBERS: float64
REGION_RATING_CLIENT: int64
REGION_RATING_CLIENT_W_CITY: int64
WEEKDAY_APPR_PROCESS_START: object
HOUR_APPR_PROCESS_START: int64
REG_REGION_NO

In [18]:
def stats(df):
    df_test_num = df.select_dtypes(include=['floating', 'integer'])
    column_names = df.columns
    unique_values = [df_test_num[each_col].nunique() for each_col in df_test_num.columns]
    data_types = [df_test_num[each_col].dtype for each_col in df_test_num.columns]
    mean =  [df_test_num[each_col].mean() for each_col in df_test_num.columns]
    median = [df_test_num[each_col].median() for each_col in df_test_num.columns]
    min_val = [df_test_num[each_col].min() for each_col in df_test_num.columns]
    max_val = [df_test_num[each_col].max() for each_col in df_test_num.columns]  
    std = [df_test_num[each_col].std() for each_col in df_test_num.columns]
    #columns = ['Column Name', 'Unique Values', 'Data Type', 'Mean', 'Median', 'Min', 'Max', 'Std']
    data_num = {'Column Name': column_names, 'Unique Values':unique_values, 'Data Type':data_types, 'Mean':mean, 'Median':median, 'Min':min_val, 'Max':max_val, 'Std':std}
    df_stats = pd.DataFrame(data=data_num)
    return df_stats.style

,Column Name,Unique Values,Data Type,Mean,Median,Min,Max,Std
0,AMT_ANNUITY,13672,float64,27108.573909,24903.000000,1615.500000,258025.500000,14493.737315
1,AMT_GOODS_PRICE,1002,float64,538396.207429,450000.000000,40500.000000,4050000.000000,369446.460540
2,OWN_CAR_AGE,62,float64,12.061091,9.000000,0.000000,91.000000,11.944812
3,CNT_FAM_MEMBERS,17,float64,2.152665,2.000000,1.000000,20.000000,0.910682
4,EXT_SOURCE_1,114584,float64,0.502130,0.505998,0.014568,0.962693,0.211062
5,EXT_SOURCE_2,119831,float64,0.514393,0.565961,0.000000,0.855000,0.191060
6,EXT_SOURCE_3,814,float64,0.510853,0.535276,0.000527,0.896010,0.194844
7,APARTMENTS_AVG,2339,float64,0.117440,0.087600,0.000000,1.000000,0.108240
8,BASEMENTAREA_AVG,3780,float64,0.088442,0.076300,0.000000,1.000000,0.082438
9,YEARS_BEGINEXPLUATATION_AVG,285,float64,0.977735,0.981600,0.000000,1.000000,0.059223


In [11]:
df_test_obj = df_test.select_dtypes(include='object')
column_names = df_test_obj.columns
unique_values = [df_test_obj[each_col].nunique() for each_col in df_test_obj.columns]
columns = ['Column Name', 'Unique Values']
data_obj = {'Column Name': column_names, 'Unique Values':unique_values}
df_test_obj_info = pd.DataFrame(data=data_obj)
df_test_obj_info


,Column Name,Unique Values
0,NAME_TYPE_SUITE,7
1,OCCUPATION_TYPE,18
2,FONDKAPREMONT_MODE,4
3,HOUSETYPE_MODE,3
4,WALLSMATERIAL_MODE,7
5,EMERGENCYSTATE_MODE,2


<hr  style= "
  display: block; 
  margin-top: 0.5em; 
  margin-bottom: 0.5em;
  margin-left: auto;
  margin-right: auto;
  border-style: inset;
  border-width: 15px "/>

## Clean dataframe --> identify data type --> force data size --> fill missing values

#### WORKING CODE: 
1) all column for data types WORKING <br>
2) all missing values filled WORKING <br>
3) all categorical columns encoded WORKING <br>
4) all columns accounted WORKING <br>

In [28]:

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
# identify if column is categorical based on number of unique values (5%)
        elif df[col].nunique()/len(df[col]) <= 0.05:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

def clean(df):
    col_int_cont=set()
    col_int_bin =set()
    col_obj=set()
    col_flt=set()
    col_obj_cat_enc=set()
    
    for col in df.columns:
        col_type = str(df[col].dtype)
# if column is OBJECT TYPE:
    if 'object' in col_type:
# Fill missing values
        df[col].fillna('unknown', inplace=True)             
        col_obj.add(col)       
    elif 'float' in col_type:
        df[col].fillna(df[col].mean(), inplace=True)
        col_flt.add(col)
    elif 'int' in col_type:
# check for binary classification for missing values to -1, and force int8
# if the missing values in a column are binary in nature (0 or 1), we do not want median, we want -1        
        if df[col].min() == 0 and df[col].max() == 1 and df[col].nunique == 2:
            df[col].fillna(-1, inplace=True)
             # Count columns: none as no integer columns in df_test
            col_int_bin.add(each_col) 
#Else if not binary@int8 then force int16, int32 or int64 and fill missing values with median
        else:
            df[col].fillna(df[col].median(), inplace=True)
# Count columns: none as no integer columns in df_test
            col_int_cont.add(each_col)
    print(f'Dataframe: {df} cleaned.')
    return df

# encode categorical column
# Working but returning column doesnt make it overwrite original df for some reason, so I used global
# but would have to figure how to use this to overwrite original df. df = df_enc.copy() also does not work
def encode(df):
    df_temp = pd.get_dummies(df.select_dtypes(include='category'))
    global df_enc
    df_enc = pd.concat([df,df_temp], axis= 1)        
    return df_enc
# Verify is not completely working as Im pretty sure my calculations of columns is off.
def verify_mv(df):
    obj_cols = len(df.select_dtypes(include='object').columns)
    num_cols = len(df.select_dtypes(include='number').columns)
    cat_cols = len(df.select_dtypes(include='category').columns)
    total_cols = len(df.columns)
    if df.isnull().sum().sum() == 0 and obj_cols + num_cols + cat_cols == total_cols:
        print( 'All missing columns seperated by data types filled and accounted for!')
        print( f'Columns stats: Total columns: {total_cols}, object columns {obj_cols} number columns:{num_cols}, categorical columns:{cat_cols} ')
    else:
        print("There were errors in processing")



#### Reduce memory usage works

In [7]:
 reduce_mem_usage(df_app_train)

Memory usage of dataframe is 286.23 MB
Memory usage after optimization is: 59.54 MB
Decreased by 79.2%


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


#### Encoding works but having issues returning the encoded df and overwriting the original

In [13]:
encode(df_app_train)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0,0,1,0,1,0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,1,0,0,0,0,0,0,1,0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,0,0,0,0,0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,0,0,1,0,1,0
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,0,0,1,0,1,0
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,0,1,0,0,1,0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0,0,1,0,1,0


#### Verifying df after forcing dtype, encoding, cleaning Its not working have to look at column count.

In [10]:
verify_mv(df_app_train)

There were errors in processing


##### Original df_app_train is not being overwrtten with encoded

In [26]:
len(df_app_train.columns), len(df_enc.columns)

(122, 262)


### Garbage collect all unused df:

In [27]:
#We will keep the dfs we use [df_app_train, df_app_train_cleaned] and dump all rest df_app_train_missing
all_df_in_mem = %who_ls DataFrame 
print(all_df_in_mem)

['df', 'df_app_train', 'df_app_tran', 'df_burr', 'df_burr_bal', 'df_cc_bal', 'df_enc', 'df_inst_pay', 'df_pos_cash', 'df_prev_app']


#### We will keep only dfs we need

In [24]:
#del [of df to dump]

73